In [1]:
%pip install semantic-link-labs
%pip install semantic-link-sempy

StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.3/699.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.9.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-fa976557-45eb-40cf-88f8-4875ae6e76a7
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
@log
def create_model_bpa_semantic_model(
    dataset: Optional[str] = icons.model_bpa_name,
    lakehouse: Optional[str | UUID] = None,
    lakehouse_workspace: Optional[str | UUID] = None,
):
    """
    Dynamically generates a Direct Lake semantic model based on the 'modelbparesults' delta table which contains the Best Practice Analyzer results.
    This semantic model used in combination with the corresponding Best Practice Analyzer report can be used to analyze multiple semantic models
    on multiple workspaces at once (and over time).

    The semantic model is always created within the same workspace as the lakehouse.

    Parameters
    ----------
    dataset : str, default='ModelBPA'
        Name of the semantic model to be created.
    lakehouse : str | uuid.UUID, default=None
        Name of the Fabric lakehouse which contains the 'modelbparesults' delta table.
        Defaults to None which resolves to the default lakehouse attached to the notebook.
    lakehouse_workspace : str | uuid.UUID, default=None
        The workspace in which the lakehouse resides.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    """

    from sempy_labs.directlake import (
        generate_shared_expression,
        add_table_to_direct_lake_semantic_model,
    )
    from sempy_labs import create_blank_semantic_model, refresh_semantic_model
    from sempy_labs.tom import connect_semantic_model

    lakehouse_workspace_id = resolve_workspace_id(workspace=lakehouse_workspace)
    (lakehouse_id, lakehouse_name) = resolve_lakehouse_name_and_id(
        lakehouse=lakehouse, workspace=lakehouse_workspace_id
    )

    # Generate the shared expression based on the lakehouse and lakehouse workspace
    expr = generate_shared_expression(
        item_name=lakehouse_name,
        item_type="Lakehouse",
        workspace=lakehouse_workspace_id,
    )

    # Create blank model
    create_blank_semantic_model(
        dataset=dataset, workspace=lakehouse_workspace_id, overwrite=True
    )

    @retry(
        sleep_time=1,
        timeout_error_message=f"{icons.red_dot} Function timed out after 1 minute",
    )
    def dyn_connect():
        with connect_semantic_model(
            dataset=dataset, readonly=True, workspace=lakehouse_workspace_id
        ) as tom:

            tom.model

    dyn_connect()
    icons.sll_tags.append("ModelBPABulk")
    table_exists = False
    with connect_semantic_model(
        dataset=dataset, readonly=False, workspace=lakehouse_workspace_id
    ) as tom:
        t_name = "BPAResults"
        t_name_full = f"'{t_name}'"
        # Create the shared expression
        if not any(e.Name == "DatabaseQuery" for e in tom.model.Expressions):
            tom.add_expression(name="DatabaseQuery", expression=expr)
        # Add the table to the model
        if any(t.Name == t_name for t in tom.model.Tables):
            table_exists = True
    if not table_exists:
        add_table_to_direct_lake_semantic_model(
            dataset=dataset,
            table_name=t_name,
            lakehouse_table_name="modelbparesults",
            workspace=lakehouse_workspace_id,
            refresh=False,
        )
    with connect_semantic_model(
        dataset=dataset, readonly=False, workspace=lakehouse_workspace_id
    ) as tom:
        # Fix column names
        for c in tom.all_columns():
            if c.Name == "Dataset_Name":
                c.Name = "Model"
            elif c.Name == "Dataset_Id":
                c.Name = "Model Id"
            elif c.Name == "Workspace_Name":
                c.Name = "Workspace"
            elif c.Name == "Capacity_Name":
                c.Name = "Capacity"
            elif c.Name == "Configured_By":
                c.Name = "Model Owner"
            elif c.Name == "URL":
                c.DataCategory = "WebURL"
            elif c.Name == "RunId":
                tom.set_summarize_by(
                    table_name=c.Parent.Name, column_name=c.Name, value="None"
                )
            c.Name = c.Name.replace("_", " ")

        # Implement pattern for base measures
        def get_expr(table_name, calculation):
            return f"IF(HASONEFILTER({table_name}[RunId]),{calculation},CALCULATE({calculation},FILTER(VALUES({table_name}[RunId]),{table_name}[RunId] = [Max Run Id])))"

        # Add measures
        int_format = "#,0"
        m_name = "Max Run Id"
        if not any(m.Name == m_name for m in tom.all_measures()):
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=f"CALCULATE(MAX({t_name_full}[RunId]),{t_name_full}[RunId])",
                format_string=int_format,
            )
        m_name = "Capacities"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS(DISTINCT({t_name_full}[Capacity]))"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Models"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS(DISTINCT({t_name_full}[Model]))"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Workspaces"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS(DISTINCT({t_name_full}[Workspace]))"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Violations"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS({t_name_full})"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Error Violations"
        if not any(m.Name == m_name for m in tom.all_measures()):
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=f'CALCULATE([Violations],{t_name_full}[Severity]="Error")',
                format_string=int_format,
            )
        m_name = "Rules Violated"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS(DISTINCT({t_name_full}[Rule Name]))"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Rule Severity"
        if not any(m.Name == m_name for m in tom.all_measures()):
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=f"IF(ISFILTERED({t_name_full}[Rule Name]),IF( HASONEVALUE({t_name_full}[Rule Name]),MIN({t_name_full}[Severity])))",
            )
        # tom.add_measure(table_name=t_name, measure_name='Rules Followed', expression="[Rules] - [Rules Violated]")

    # Refresh the model
    refresh_semantic_model(dataset=dataset, workspace=lakehouse_workspace_id)


StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 10, Finished, Available, Finished)

NameError: name 'log' is not defined

In [5]:
import sempy.fabric as fabric
import sempy_labs as labs

StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 13, Finished, Available, Finished)

In [22]:
with connect_semantic_model(dataset=dataset, readonly=True, workspace=workspace) as tom:
    database = tom.model.Database
    print(database)


StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 30, Finished, Available, Finished)

fse_golden-30min


In [ ]:
dir(database)

StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, -1, Cancelled, , Cancelled)

In [ ]:
#Annotations
with connect_semantic_model(dataset=dataset, readonly=True, workspace=workspace) as tom:
    for annotation in tom.model.get_Database:
        print(f"Annotation Name: {annotation.Name}")
        print(f"Annotation Value: {annotation.Value}")
        print("-" * 40)


In [21]:
#Annotations
with connect_semantic_model(dataset=dataset, readonly=True, workspace=workspace) as tom:
    for annotation in tom.model.Annotations:
        print(f"Annotation Name: {annotation.Name}")
        print(f"Annotation Value: {annotation.Value}")
        print("-" * 40)


StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 29, Finished, Available, Finished)

Annotation Name: __PBI_TimeIntelligenceEnabled
Annotation Value: 0
----------------------------------------
Annotation Name: PBIDesktopVersion
Annotation Value: 2.137.1102.0 (24.10)
----------------------------------------
Annotation Name: PBI_QueryOrder
Annotation Value: ["DimAD","DimDate","DimDealer","DimPart","FactADRetail","FactAMDRetail","Retail","RangeStart","RangeEnd","FactDespatch","Billing, Dispatch & Booking","FactBooking","FactRetailMonthPlan","Last Refresh","DimBusinessModel","LocationMapping","ProductMapping","3W Dom Retails","ZMList","TMList","AMList","DealerList","StateList","SparkURL","FactRetailADMonthPlan","FactBilling","workspaceName","dataflowName","DataflowNamerole","FactFestiveRetailsHourly","Date&Inv_Hour","FactRtliQubeDayPlan","FactAMDFundingcc","FactEMOSPlanFlag","DimFlagtable","FactADFundingcc","FactFestiveHourlyCumulativeRetails0Hour","FactFestiveHourlyCumulativeRetails"]
----------------------------------------
Annotation Name: __TEdtr
Annotation Value: 1
--

In [20]:
with connect_semantic_model(dataset=dataset, readonly=True, workspace=workspace) as tom:
    print(tom.model.Annotations)
    # for t in tom.model.Annotations:
        # print(t.get_Model)
        # print(dir(t))

StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 28, Finished, Available, Finished)

Microsoft.AnalysisServices.Tabular.ModelAnnotationCollection


In [16]:
dir(tom.model)

StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 24, Finished, Available, Finished)

['AnalyticsAIMetadata',
 'Annotations',
 'ApplyAutomaticAggregations',
 'ApplyPerformanceRecommendations',
 'ApplyRefreshPolicies',
 'AutomaticAggregationOptions',
 'BindingInfoCollection',
 'Clone',
 'Collation',
 'CopyFrom',
 'CopyTo',
 'Culture',
 'Cultures',
 'DataAccessOptions',
 'DataSourceDefaultMaxConnections',
 'DataSourceVariablesOverrideBehavior',
 'DataSources',
 'Database',
 'DefaultDataView',
 'DefaultMeasure',
 'DefaultMode',
 'DefaultPowerBIDataSourceVersion',
 'Description',
 'DirectLakeBehavior',
 'DisableAutoExists',
 'DiscourageCompositeModels',
 'DiscourageImplicitMeasures',
 'DiscourageReportMeasures',
 'Equals',
 'ExcludedArtifacts',
 'ExecuteXmla',
 'Expressions',
 'ExtendedProperties',
 'Finalize',
 'ForceUniqueNames',
 'Functions',
 'GetCompatibilityRequirementByMembers',
 'GetHashCode',
 'GetType',
 'HasLocalChanges',
 'IsRemoved',
 'MAttributes',
 'MaxParallelismPerQuery',
 'MaxParallelismPerRefresh',
 'MemberwiseClone',
 'Model',
 'ModifiedTime',
 'Name',
 

In [2]:
import sempy
sempy.fabric._client._utils._init_analysis_services()
import Microsoft.AnalysisServices.Tabular as TOM
import pandas as pd

StatementMeta(, 6154b121-9922-4d54-b73b-4d58a9cf43a0, 10, Finished, Available, Finished)

In [3]:
import sempy_labs as labs
from sempy_labs.tom import connect_semantic_model

StatementMeta(, 6154b121-9922-4d54-b73b-4d58a9cf43a0, 11, Finished, Available, Finished)

In [11]:
dataset = 'fse_golden-30min' # Enter dataset name
workspace = 'BI Governance'

StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 19, Finished, Available, Finished)

In [15]:
with connect_semantic_model(dataset=dataset, readonly=False, workspace=workspace) as tom:
    tom.set_vertipaq_annotations()

    for t in tom.model.Tables:
        rc = tom.row_count(object = t)
        print(f"{t.Name}  : {str(rc)}")
        for c in t.Columns:
            col_size = tom.total_size(object=c)
            print(labs.format_dax_object_name(t.Name, c.Name) + ' : ' + str(col_size))

StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 23, Finished, Available, Finished)

DimAD  : 8381
'DimAD'[RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61] : 128
'DimAD'[AD ID] : 248852
'DimAD'[AMD ID] : 62108
'DimAD'[AD Name] : 308463
'DimAD'[AD] : 464795
'DimAD'[City] : 361124
DimDate  : 2922
'DimDate'[RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61] : 128
'DimDate'[Datekey] : 135076
'DimDate'[Year] : 1664
'DimDate'[Month] : 23112
'DimDate'[Start Of Month] : 6628
'DimDate'[End Of Month] : 6640
'DimDate'[Date] : 135076
'DimDate'[StartDay] : 1520
'DimDate'[EndDay] : 1520
'DimDate'[FY] : 17704
'DimDate'[Month Number] : 3088
'DimDate'[Day] : 3724
'DimDate'[CurrentMonth] : 1528
'DimDate'[key] : 19768
DimDealer  : 7558
'DimDealer'[RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61] : 128
'DimDealer'[Active] : 17616
'DimDealer'[Pin Code] : 99468
'DimDealer'[City] : 76772
'DimDealer'[Territory] : 28964
'DimDealer'[Area] : 21424
'DimDealer'[State] : 20580
'DimDealer'[Zone] : 19698
'DimDealer'[Country ID] : 1536
'DimDealer'[Country] : 17386
'DimDealer'[Dealer] : 476005
'DimDealer'[D

OperationException: Failed to save modifications to the server. Error returned: 'It looks like you can't make changes to this dataset. Please contact a capacity administrator to make sure XMLA read/write support is enabled in the capacity settings on the Power BI Premium capacity, and then try again. For additional information, see 'XMLA read/write support' in the product documentation.

Technical Details:
RootActivityId: 685da818-c780-4b5b-b69b-c8c918531b61
Date (UTC): 4/28/2025 6:17:49 AM
'.
   at Microsoft.AnalysisServices.Tabular.Model.SaveChangesImpl(SaveFlags flags, Int32 maxParallelism)
   at Microsoft.AnalysisServices.Tabular.Model.SaveChanges()

In [14]:
import Microsoft.AnalysisServices.Tabular as TOM
from sempy.fabric._client._utils import _init_analysis_services
from sempy.fabric import workspace  # This is correct!!

# Initialize TOM environment
_init_analysis_services()

# Get the workspace and dataset
ws = workspace.get_workspace("BI Governance")
ds = workspace.get_dataset(ws.id, "fse_golden-30min")
model = ds.model  # This is a TOM.Model

# Loop through tables and partitions
for table in model.tables:
    print(f"Table: {table.name}")
    for partition in table.partitions:
        source = partition.source
        if isinstance(source, TOM.MPartitionSource):
            print(f"  Partition: {partition.name}")
            print(f"  M Query:\n{source.Expression}")
        else:
            print(f"  Partition {partition.name} is not an MPartitionSource.")


StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 22, Finished, Available, Finished)

ImportError: cannot import name 'workspace' from 'sempy.fabric' (/nfs4/pyenv-a6d5851e-1161-460d-a666-698887dea8b7/lib/python3.10/site-packages/sempy/fabric/__init__.py)

In [12]:
with connect_semantic_model(dataset=dataset, readonly=True, workspace=workspace) as tom:
    for p in tom.all_partitions():
        print(p.Name)

StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 20, Finished, Available, Finished)

DimAD-c3141ce4-53eb-4f09-95d4-46520379515b
DimDate-0a638352-c190-46c4-ab3c-736f66f3d34a
DimDealer-77f2cd30-7c73-4b34-8347-e5e84371c9d4
DimPart-c483ef97-3005-49a4-bd55-7da65e2097a1
FactADRetail-453d5af1-aadf-46fe-b94f-969d3c22af8f
FactAMDRetail-c51fe981-fde8-44b4-9f12-f26a499df35b
Retail-a342db34-e5fa-4520-bf92-1e731dc118fe
FactDespatch-1b6abf6a-8517-498f-9ab9-4b507c83d3c7
FactBilling-1a664c23-9f3f-4d3a-8fca-edbaa31eeff7
Billing, Dispatch & Booking-cf1b8dcb-d754-4c46-926b-cecc6b336f87
FactBooking-cb11ce23-72c3-42c1-b46f-ae3e5ff833e6
FactRetailMonthPlan-dc9324a2-a41f-4a71-8d75-11289d52e86f
Last Refresh-c7d416f7-d9ba-45bf-a9f0-28686e08c46e
DimBusinessModel-06c903f9-c5d8-46b4-ae3d-07be0efd7505
LocationMapping-5aa482d6-ad8d-4532-8df3-48a9dd637173
ProductMapping-f8fab31d-8e88-460d-8465-2f80527ad80f
ZMList-a2a9aac9-42bb-4608-84bb-bdf0ff83d9f7
TMList-c8660dfb-1357-475d-aff4-d00f93dec553
AMList-7dbb0468-bf65-4543-9533-e012e74659fb
DealerList-710fcf90-8d4c-4de9-86cf-a5a600211389
StateList-e61bc0

In [4]:
dir(TOM)

StatementMeta(, 15ccecaa-3527-4fdc-a008-58abcc49fbd0, 12, Finished, Available, Finished)

['AggregateFunction',
 'Alignment',
 'AlternateOf',
 'AlternateOfAnnotationCollection',
 'AnalyticsAIMetadata',
 'AnalyticsAIMetadataCollection',
 'Annotation',
 'Assembly',
 'AttributeHierarchy',
 'AttributeHierarchyAnnotationCollection',
 'AttributeHierarchyExtendedPropertyCollection',
 'AuthenticationKind',
 'AutomaticAggregationOptions',
 'BasicRefreshPolicy',
 'BindingInfo',
 'BindingInfoAnnotationCollection',
 'BindingInfoCollection',
 'BindingInfoExtendedPropertyCollection',
 'BindingInfoType',
 'CalculatedColumn',
 'CalculatedPartitionSource',
 'CalculatedTableColumn',
 'CalculationGroup',
 'CalculationGroupAnnotationCollection',
 'CalculationGroupExpression',
 'CalculationGroupSelectionMode',
 'CalculationGroupSource',
 'CalculationItem',
 'CalculationItemCollection',
 'Calendar',
 'CalendarCollection',
 'ChangedProperty',
 'ClrAssembly',
 'Column',
 'ColumnAnnotationCollection',
 'ColumnChangedPropertyCollection',
 'ColumnCollection',
 'ColumnExtendedPropertyCollection',
 'Co

In [5]:
res = labs.list_data_pipelines()

StatementMeta(, 98e0d5f5-6659-4821-8803-46c64c3d036d, 13, Finished, Available, Finished)

In [6]:
res 

StatementMeta(, 98e0d5f5-6659-4821-8803-46c64c3d036d, 14, Finished, Available, Finished)

,Data Pipeline Name,Data Pipeline ID,Description
0,pipeline1,f30c12b3-b45d-44f0-a9ec-88acaa9811aa,
1,Load Data,d1e21f10-5157-4087-9bf1-af2ae88335ae,
2,pl_LoadDim,b042114e-728e-407b-add9-76b36b107b10,
3,learning,e060e855-4d34-4c61-9e9c-dfaa762efeb1,
4,Load_Items_E2E,0a09bc7d-6c20-424c-ac19-c3af22575d9e,Imported from fab
5,Load_Activities_E2E,48fa2238-b5a3-4384-9a76-9cc81e4b9f96,Imported from fab
6,Load_Capacities_E2E,ed81ab77-f551-47e7-a021-126fb1791480,Imported from fab
7,Load_Capacity_Refreshables_E2E,74a2ab42-aebc-44cd-800f-c8cc32bed861,Imported from fab
8,Load_PBI_Workspaces_E2E,4bca9ec0-397d-42fc-bfd3-94b27d9709aa,Imported from fab
9,Load_Inventory_E2E,c141ffaa-50bc-441f-8724-24eaf83df6c6,Imported from fab


In [20]:
help(labs.list_data_pipelines)


StatementMeta(, 5b0f876c-3857-479a-9aef-b2194ad3d831, 28, Finished, Available, Finished)

Help on function list_data_pipelines in module sempy_labs._data_pipelines:

list_data_pipelines(workspace: Union[str, uuid.UUID, NoneType] = None) -> pandas.core.frame.DataFrame
    Shows the data pipelines within a workspace.
    
    This is a wrapper function for the following API: `Items - List Data Pipelines <https://learn.microsoft.com/rest/api/fabric/datapipeline/items/list-data-pipelines>`_.
    
    Parameters
    ----------
    workspace : str | uuid.UUID, default=None
        The Fabric workspace name or ID.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    
    Returns
    -------
    pandas.DataFrame
        A pandas dataframe showing the data pipelines within a workspace.



In [45]:
import sempy_labs as labs

StatementMeta(, f5d04f59-1528-41b5-9775-3704c80d31e9, 53, Finished, Available, Finished)

In [50]:
res = labs.list_dataflows(workspace="SST D&AI")

StatementMeta(, f5d04f59-1528-41b5-9775-3704c80d31e9, 58, Finished, Available, Finished)

In [51]:
display(res)

StatementMeta(, f5d04f59-1528-41b5-9775-3704c80d31e9, 59, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 691c3dd4-63a8-4511-a6f4-2fb17b10162d)

In [47]:
help(labs.list_dataflows)

StatementMeta(, f5d04f59-1528-41b5-9775-3704c80d31e9, 55, Finished, Available, Finished)

Help on function list_dataflows in module sempy_labs._dataflows:

list_dataflows(workspace: Union[str, uuid.UUID, NoneType] = None)
    Shows a list of all dataflows which exist within a workspace.
    
    Parameters
    ----------
    workspace : str | uuid.UUID, default=None
        The Fabric workspace name or ID.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    
    Returns
    -------
    pandas.DataFrame
        A pandas dataframe showing the dataflows which exist within a workspace.



In [4]:
fabric.list_gateways()

StatementMeta(, f5d04f59-1528-41b5-9775-3704c80d31e9, 12, Finished, Available, Finished)

,Gateway Id,Gateway Name,Gateway Type


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, current_date
from delta.tables import DeltaTable
import json
import pandas as pd
import requests
import concurrent.futures
import time
import msal
import datetime

StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 11, Finished, Available, Finished)

In [4]:
import sempy_labs as labs
from sempy_labs import migration, directlake, admin, graph, vertipaq_analyzer
from sempy_labs import lakehouse as lake
from sempy_labs import report as rep
from sempy_labs.tom import connect_semantic_model
from sempy_labs.report import ReportWrapper
import sempy
sempy.fabric._client._utils._init_analysis_services()
import Microsoft.AnalysisServices.Tabular as TOM


StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 12, Finished, Available, Finished)

In [5]:
import sempy.fabric as fabric
import pandas as pd
import datetime


from sempy_labs._helper_functions import (
    resolve_lakehouse_name,
    save_as_delta_table,
    resolve_workspace_capacity,
    retry,
    _get_column_aggregate,
    resolve_workspace_id,
    resolve_lakehouse_name_and_id,
)
from sempy_labs.lakehouse import (
    get_lakehouse_tables,
    lakehouse_attached,
)

from sempy_labs._model_bpa import run_model_bpa
from typing import Optional, List
from sempy._utils._log import log
import sempy_labs._icons as icons
from uuid import UUID

StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 13, Finished, Available, Finished)

In [6]:
@log
def run_power_insighter(
    language: Optional[str] = None,
    workspace: Optional[str | UUID | List[str | UUID]] = None,
    skip_models: Optional[str | List[str]] = ["ModelBPA", "Fabric Capacity Metrics"],
    skip_models_in_workspace: Optional[dict] = None,
):
    """
    Runs all the sempy functions and then it will create a Semantic Model where you will get all the articats and directly you can do reporting

    Parameters
    ----------
    language : str, default=None
        The language (code) in which the rules will appear. For example, specifying 'it-IT' will show the Rule Name, Category and Description in Italian.
        Defaults to None which resolves to English.
    workspace : str | uuid.UUID | List[str | uuid.UUID], default=None
        The workspace or list of workspaces to scan. Supports both the workspace name and the workspace id.
        Defaults to None which scans all accessible workspaces.
    skip_models : str | List[str], default=['ModelBPA', 'Fabric Capacity Metrics']
        The semantic models to always skip when running this analysis.
    skip_models_in_workspace : dict, default=None
        A dictionary showing specific semantic models within specific workspaces to skip. See the example below:
        {
            "Workspace A": ["Dataset1", "Dataset2"],
            "Workspace B": ["Dataset5", "Dataset 8"],
        }
    """
    if not lakehouse_attached():
        raise ValueError(
            f"{icons.red_dot} No lakehouse is attached to this notebook. Must attach a lakehouse to the notebook."
        )

    if isinstance(skip_models, str):
        skip_models = [skip_models]

    skip_models.extend(["ModelBPA", "Fabric Capacity Metrics"])

    now = datetime.datetime.now()

    #Pulling all the workspaces details
    dfW = fabric.list_workspaces("type ne 'AdminInsights'")
    if workspace is None:
        dfW_filt = dfW.copy()
    else:
        dfW_filt = dfW[(dfW["Name"].isin(workspace)) | (dfW["Id"].isin(workspace))]

    if dfW_filt.empty:
        raise ValueError(
            f"{icons.red_dot} There are no valid workspaces to assess. This is likely due to not having proper permissions to the workspace(s) entered in the 'workspace' parameter."
        )

    # Convert 'Name' column to list
    workspaces_list = dfW_filt['Id'].tolist()



    all_dfD = pd.DataFrame()
    for _, r in dfW_filt.iterrows():
        wksp = r["Name"]
        wksp_id = r["Id"]
        # capacity_id, capacity_name = resolve_workspace_capacity(workspace=wksp)

        # df = pd.DataFrame(columns=list(icons.bpa_schema.keys()))

        # Get dataset details
        dfD = fabric.list_datasets(workspace=wksp, mode="rest")

        #Adding Workspace ID to the dataframe
        dfD["Workspace Id"] = wksp_id

        # Append to the master DataFrame
        all_dfD = pd.concat([all_dfD, dfD], ignore_index=True)

    #Pulling all apps 
    dfApps = fabric.list_apps()

    #Pulling all capacities
    dfCapacities = fabric.list_capacities()

    #List DataFlows
    dfDataFlows = fabric.list_dataflows()

    return dfW

StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 14, Finished, Available, Finished)

In [15]:
@log
def run_power_insighter(
    language: Optional[str] = None,
    workspace: Optional[str | UUID | List[str | UUID]] = None,
    skip_models: Optional[str | List[str]] = ["ModelBPA", "Fabric Capacity Metrics"],
    skip_models_in_workspace: Optional[dict] = None,
):
    """
    Runs all the sempy functions and then it will create a Semantic Model where you will get all the articats and directly you can do reporting

    Parameters
    ----------
    language : str, default=None
        The language (code) in which the rules will appear. For example, specifying 'it-IT' will show the Rule Name, Category and Description in Italian.
        Defaults to None which resolves to English.
    workspace : str | uuid.UUID | List[str | uuid.UUID], default=None
        The workspace or list of workspaces to scan. Supports both the workspace name and the workspace id.
        Defaults to None which scans all accessible workspaces.
    skip_models : str | List[str], default=['ModelBPA', 'Fabric Capacity Metrics']
        The semantic models to always skip when running this analysis.
    skip_models_in_workspace : dict, default=None
        A dictionary showing specific semantic models within specific workspaces to skip. See the example below:
        {
            "Workspace A": ["Dataset1", "Dataset2"],
            "Workspace B": ["Dataset5", "Dataset 8"],
        }
    """
    if not lakehouse_attached():
        raise ValueError(
            f"{icons.red_dot} No lakehouse is attached to this notebook. Must attach a lakehouse to the notebook."
        )

    if isinstance(skip_models, str):
        skip_models = [skip_models]

    skip_models.extend(["ModelBPA", "Fabric Capacity Metrics"])

    now = datetime.datetime.now()

    #Pulling all the workspaces details
    dfW = fabric.list_workspaces("type ne 'AdminInsights'")
    if workspace is None:
        dfW_filt = dfW.copy()
    else:
        dfW_filt = dfW[(dfW["Name"].isin(workspace)) | (dfW["Id"].isin(workspace))]

    if dfW_filt.empty:
        raise ValueError(
            f"{icons.red_dot} There are no valid workspaces to assess. This is likely due to not having proper permissions to the workspace(s) entered in the 'workspace' parameter."
        )

    # Convert 'Name' column to list
    workspaces_list = dfW_filt['Id'].tolist()



    all_dfD = pd.DataFrame()
    all_DatasetSourceDetails = pd.DataFrame()
    all_dfApps = pd.DataFrame()
    all_dfDataFlows = pd.DataFrame()

    all_dfCapacities = fabric.list_capacities()
    for _, r in dfW_filt.iterrows():
        wksp = r["Name"]
        wksp_id = r["Id"]
        # capacity_id, capacity_name = resolve_workspace_capacity(workspace=wksp)

        # df = pd.DataFrame(columns=list(icons.bpa_schema.keys()))

        # Get dataset details
        dfD = fabric.list_datasets(workspace=wksp, mode="rest")

        #Adding Workspace ID to the dataframe
        dfD["Workspace Id"] = wksp_id
        print(type(dfD))

        # Append to the master DataFrame
        all_dfD = pd.concat([all_dfD, dfD], ignore_index=True)

        #Pulling all apps 
        all_dfApps = fabric.list_apps()

        #Pulling all capacities
        

        #List DataFlows
        # all_dfDataFlows = fabric.list_dataflows()

    return dfD

StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 23, Finished, Available, Finished)

In [16]:
test = run_power_insighter()

StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 24, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [8]:
test

StatementMeta(, 56c09688-a11b-4c7d-ae40-74848da01bb0, 16, Finished, Available, Finished)

,Id,Is Read Only,Is On Dedicated Capacity,Capacity Id,Default Dataset Storage Format,Type,Name
0,35060111-ead1-48c7-90ad-6470f10fc59e,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,3. Domestic 2W Marketing
1,fd2fc524-9196-41e8-9ecb-2f4e8e8ded50,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,1. Domestic 2W Sales
2,e86870f9-b5ef-4562-824c-4fb553d63e7b,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,5. Domestic 2W Digital Assets
3,83ec31a1-2ca3-46fe-a5f8-2bcc8e70f8cd,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,2. Domestic 2W Dealer Management
4,13a5a575-b5bb-4e9f-8e12-499d4af44889,False,False,NaN,NaN,Workspace,Development_Workspace
5,7acb5d88-77de-4762-bb6d-57b1ce0afa46,False,False,NaN,NaN,Workspace,BI QC
6,228db8e8-6203-4ba8-b406-7b1da29c34c1,False,True,76f4499e-05ff-44a9-8c2f-323ee14ea1a7,Small,Workspace,P1 Priority Datasets
7,a912e8a5-7138-4452-88a1-5c7df2c99a5a,False,False,NaN,NaN,Workspace,Beta Testing Workspace
8,8b09a058-33d3-4ef3-a8dd-f4bc5e740652,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,7. International Business
9,8ba77fd7-a1ef-4837-8f15-915ba50d0f4a,False,True,517b62f7-2477-4809-a9d5-bfbec118442c,Small,Workspace,BI Governance


In [73]:
fabric.list_datasources(workspace = "BI QC")

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 81, Finished, Available, Finished)

TypeError: missing a required argument: 'dataset'

In [72]:
help(fabric.list_datasources)

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 80, Finished, Available, Finished)

Help on function list_datasources in module sempy.fabric._flat_list_datasources:

list_datasources(dataset: Union[str, uuid.UUID], additional_xmla_properties: Union[str, List[str], NoneType] = None, workspace: Union[str, uuid.UUID, NoneType] = None) -> pandas.core.frame.DataFrame
    List all datasources in a dataset.
    
    Parameters
    ----------
    dataset : str or uuid.UUID
        Name or UUID of the dataset.
    additional_xmla_properties : str or List[str], default=None
        Additional XMLA `datasource <https://learn.microsoft.com/en-us/dotnet/api/microsoft.analysisservices.tabular.datasource?view=analysisservices-dotnet>`_
        properties to include in the returned dataframe.
    workspace : str or uuid.UUID, default=None
        The Fabric workspace name or UUID object containing the workspace ID.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    
   

In [66]:
display(test)

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 74, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 90c69a54-aa3c-47f7-aa17-5f4d3f7f817c)

In [60]:
help(fabric.list_datasets)

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 68, Finished, Available, Finished)

Help on function list_datasets in module sempy.fabric._flat:

list_datasets(workspace: Union[str, uuid.UUID, NoneType] = None, mode: str = 'xmla', additional_xmla_properties: Union[str, List[str], NoneType] = None) -> pandas.core.frame.DataFrame
    List datasets in a `Fabric workspace <https://learn.microsoft.com/en-us/fabric/get-started/workspaces>`_.
    
    Parameters
    ----------
    workspace : str or uuid.UUID, default=None
        The Fabric workspace name or UUID object containing the workspace ID.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    mode : str, default="xmla"
        Whether to use the XMLA "xmla" or REST API "rest".
        See `REST docs <https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/get-datasets>`_ for returned fields.
    additional_xmla_properties : str or List[str], default=None
        Additional XMLA `model <https://learn

In [57]:
print(test)

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 65, Finished, Available, Finished)

['7acb5d88-77de-4762-bb6d-57b1ce0afa46', '8ba77fd7-a1ef-4837-8f15-915ba50d0f4a']


In [24]:
dfW = fabric.list_workspaces("type ne 'AdminInsights'")

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 32, Finished, Available, Finished)

In [25]:
dfW

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 33, Finished, Available, Finished)

,Id,Is Read Only,Is On Dedicated Capacity,Capacity Id,Default Dataset Storage Format,Type,Name
0,35060111-ead1-48c7-90ad-6470f10fc59e,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,3. Domestic 2W Marketing
1,fd2fc524-9196-41e8-9ecb-2f4e8e8ded50,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,1. Domestic 2W Sales
2,e86870f9-b5ef-4562-824c-4fb553d63e7b,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,5. Domestic 2W Digital Assets
3,83ec31a1-2ca3-46fe-a5f8-2bcc8e70f8cd,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,2. Domestic 2W Dealer Management
4,13a5a575-b5bb-4e9f-8e12-499d4af44889,False,False,NaN,NaN,Workspace,Development_Workspace
5,7acb5d88-77de-4762-bb6d-57b1ce0afa46,False,False,NaN,NaN,Workspace,BI QC
6,228db8e8-6203-4ba8-b406-7b1da29c34c1,False,True,76f4499e-05ff-44a9-8c2f-323ee14ea1a7,Small,Workspace,P1 Priority Datasets
7,a912e8a5-7138-4452-88a1-5c7df2c99a5a,False,False,NaN,NaN,Workspace,Beta Testing Workspace
8,8b09a058-33d3-4ef3-a8dd-f4bc5e740652,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,7. International Business
9,8ba77fd7-a1ef-4837-8f15-915ba50d0f4a,False,True,517b62f7-2477-4809-a9d5-bfbec118442c,Small,Workspace,BI Governance


In [20]:
dir(fabric)

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 28, Finished, Available, Finished)

['CognitiveServiceRestClient',
 'DataCategory',
 'FabricDataFrame',
 'FabricRestClient',
 'FabricSeries',
 'MetadataKeys',
 'ModelCalcDependencies',
 'PowerBIRestClient',
 'RefreshExecutionDetails',
 'TOMWrapper',
 'Trace',
 'TraceConnection',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '_bpa',
 '_cache',
 '_client',
 '_datacategory',
 '_dataframe',
 '_environment',
 '_flat',
 '_flat_list_annotations',
 '_flat_list_apps',
 '_flat_list_calculation_items',
 '_flat_list_columns',
 '_flat_list_dataflows',
 '_flat_list_datasources',
 '_flat_list_gateways',
 '_flat_list_hierarchies',
 '_flat_list_partitions',
 '_flat_list_perspectives',
 '_flat_list_relationships',
 '_metadatakeys',
 '_token_provider',
 '_trace',
 '_trace_evaluate_dax',
 '_utils',
 '_vertipaq',
 'connect_semantic_model',
 'create_lakehouse',
 'create_notebook',
 'create_tom_server',
 'create_trace_connection',
 'create_workspac

In [19]:
display(test)

StatementMeta(, 366858a0-394d-4996-a191-60716f8dbf4c, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 429c2d38-c019-4c25-8d97-5b13799106d7)

In [51]:
import pandas as pd
from sempy_labs._helper_functions import (
    resolve_workspace_name_and_id,
    _is_valid_uuid,
    _update_dataframe_datatypes,
    _base_api,
    _create_dataframe,
    resolve_workspace_name,
)
from typing import Optional, Tuple, List
import sempy_labs._icons as icons
from uuid import UUID
import sempy.fabric as fabric


def list_dataflows(workspace: Optional[str | UUID | List[str | UUID]] = None):
    """
    Shows a list of all dataflows which exist within specified or all workspaces .

    Parameters
    ----------
    workspace : str | uuid.UUID | List[str | uuid.UUID], default=None
        The workspace or list of workspaces to scan. Supports both the workspace name and the workspace id.
        Defaults to None which scans all accessible workspaces.

    Returns
    -------
    pandas.DataFrame
        A pandas dataframe showing the dataflows which exist within a workspace.
    """

    #Pulling all the workspaces details
    dfW = fabric.list_workspaces("type ne 'AdminInsights'")
    if workspace is None:
        dfW_filt = dfW.copy()
    else:
        if isinstance(workspace, str):
            workspace = [workspace]
        dfW_filt = dfW[(dfW["Name"].isin(workspace)) | (dfW["Id"].isin(workspace))]

    if dfW_filt.empty:
        raise ValueError(
            f"{icons.red_dot} There are no valid workspaces to assess. This is likely due to not having proper permissions to the workspace(s) entered in the 'workspace' parameter."
        )

    # Convert 'Name' column to list
    workspaces_list = dfW_filt['Id'].tolist()


    columns = {
        "Workspace Id": "string",
        "Workspace Name": "string",
        "Dataflow Id": "string",
        "Dataflow Name": "string",
        "Configured By": "string",
        "Users": "string",
        "Generation": "int",
    }
    df = _create_dataframe(columns=columns)

    
    for workspace_id in workspaces_list:
        (workspace_name, workspace_id) = resolve_workspace_name_and_id(workspace_id)
        response = _base_api(request=f"/v1.0/myorg/groups/{workspace_id}/dataflows")

        data = []  # Collect rows here

        for v in response.json().get("value", []):
            new_data = {
                "Workspace Id": workspace_id,
                "Workspace Name": workspace_name,
                "Dataflow Id": v.get("objectId"),
                "Dataflow Name": v.get("name"),
                "Configured By": v.get("configuredBy"),
                "Users": v.get("users", []),
                "Generation": v.get("generation"),
            }
            data.append(new_data)

        if data:
            df_temp = pd.DataFrame(data)
            _update_dataframe_datatypes(dataframe=df_temp, column_map=columns)
            df = pd.concat([df, df_temp], ignore_index=True)

    return df

StatementMeta(, 5b0f876c-3857-479a-9aef-b2194ad3d831, 59, Finished, Available, Finished)

In [48]:
workspace = ["SST D&AI","8b09a058-33d3-4ef3-a8dd-f4bc5e740652"]

StatementMeta(, 5b0f876c-3857-479a-9aef-b2194ad3d831, 56, Finished, Available, Finished)

In [54]:
test = list_dataflows(workspace = "SST D&AI")

StatementMeta(, 5b0f876c-3857-479a-9aef-b2194ad3d831, 62, Finished, Available, Finished)

In [55]:
display(test)

StatementMeta(, 5b0f876c-3857-479a-9aef-b2194ad3d831, 63, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ba4d1944-4136-4157-8a51-0c7385fc5815)

In [75]:
test = list_dataflows(workspace = ["SST D&AI","8b09a058-33d3-4ef3-a8dd-f4bc5e740652"])

StatementMeta(, f5d04f59-1528-41b5-9775-3704c80d31e9, 83, Finished, Available, Finished)

In [4]:
import sempy.fabric as fabric
import pandas as pd
import datetime
from sempy_labs._helper_functions import (
    save_as_delta_table,
    resolve_workspace_capacity,
    retry,
    _get_column_aggregate,
    resolve_workspace_id,
    resolve_lakehouse_name_and_id,
)
from sempy_labs.lakehouse import (
    get_lakehouse_tables,
    lakehouse_attached,
)
from sempy_labs._model_bpa import run_model_bpa
from typing import Optional, List
from sempy._utils._log import log
import sempy_labs._icons as icons
from uuid import UUID

StatementMeta(, 30d88d90-4800-4e92-92f1-49c9ba35c7de, 12, Finished, Available, Finished)

In [5]:
@log
def create_model_bpa_semantic_model(
    dataset: Optional[str] = icons.model_bpa_name,
    lakehouse: Optional[str | UUID] = None,
    lakehouse_workspace: Optional[str | UUID] = None,
):
    """
    Dynamically generates a Direct Lake semantic model based on the 'modelbparesults' delta table which contains the Best Practice Analyzer results.
    This semantic model used in combination with the corresponding Best Practice Analyzer report can be used to analyze multiple semantic models
    on multiple workspaces at once (and over time).

    The semantic model is always created within the same workspace as the lakehouse.

    Parameters
    ----------
    dataset : str, default='ModelBPA'
        Name of the semantic model to be created.
    lakehouse : str | uuid.UUID, default=None
        Name of the Fabric lakehouse which contains the 'modelbparesults' delta table.
        Defaults to None which resolves to the default lakehouse attached to the notebook.
    lakehouse_workspace : str | uuid.UUID, default=None
        The workspace in which the lakehouse resides.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    """

    from sempy_labs.directlake import (
        generate_shared_expression,
        add_table_to_direct_lake_semantic_model,
    )
    from sempy_labs import create_blank_semantic_model, refresh_semantic_model
    from sempy_labs.tom import connect_semantic_model

    lakehouse_workspace_id = resolve_workspace_id(workspace=lakehouse_workspace)
    (lakehouse_id, lakehouse_name) = resolve_lakehouse_name_and_id(
        lakehouse=lakehouse, workspace=lakehouse_workspace_id
    )

    # Generate the shared expression based on the lakehouse and lakehouse workspace
    expr = generate_shared_expression(
        item_name=lakehouse_name,
        item_type="Lakehouse",
        workspace=lakehouse_workspace_id,
    )

    # Create blank model
    create_blank_semantic_model(
        dataset=dataset, workspace=lakehouse_workspace_id, overwrite=True
    )

    @retry(
        sleep_time=1,
        timeout_error_message=f"{icons.red_dot} Function timed out after 1 minute",
    )
    def dyn_connect():
        with connect_semantic_model(
            dataset=dataset, readonly=True, workspace=lakehouse_workspace_id
        ) as tom:

            tom.model

    dyn_connect()
    icons.sll_tags.append("ModelBPABulk")
    table_exists = False
    with connect_semantic_model(
        dataset=dataset, readonly=False, workspace=lakehouse_workspace_id
    ) as tom:
        t_name = "BPAResults"
        t_name_full = f"'{t_name}'"
        # Create the shared expression
        if not any(e.Name == "DatabaseQuery" for e in tom.model.Expressions):
            tom.add_expression(name="DatabaseQuery", expression=expr)
        # Add the table to the model
        if any(t.Name == t_name for t in tom.model.Tables):
            table_exists = True
    if not table_exists:
        add_table_to_direct_lake_semantic_model(
            dataset=dataset,
            table_name=t_name,
            lakehouse_table_name="modelbparesults",
            workspace=lakehouse_workspace_id,
            refresh=False,
        )
    with connect_semantic_model(
        dataset=dataset, readonly=False, workspace=lakehouse_workspace_id
    ) as tom:
        # Fix column names
        for c in tom.all_columns():
            if c.Name == "Dataset_Name":
                c.Name = "Model"
            elif c.Name == "Dataset_Id":
                c.Name = "Model Id"
            elif c.Name == "Workspace_Name":
                c.Name = "Workspace"
            elif c.Name == "Capacity_Name":
                c.Name = "Capacity"
            elif c.Name == "Configured_By":
                c.Name = "Model Owner"
            elif c.Name == "URL":
                c.DataCategory = "WebURL"
            elif c.Name == "RunId":
                tom.set_summarize_by(
                    table_name=c.Parent.Name, column_name=c.Name, value="None"
                )
            c.Name = c.Name.replace("_", " ")

        # Implement pattern for base measures
        def get_expr(table_name, calculation):
            return f"IF(HASONEFILTER({table_name}[RunId]),{calculation},CALCULATE({calculation},FILTER(VALUES({table_name}[RunId]),{table_name}[RunId] = [Max Run Id])))"

        # Add measures
        int_format = "#,0"
        m_name = "Max Run Id"
        if not any(m.Name == m_name for m in tom.all_measures()):
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=f"CALCULATE(MAX({t_name_full}[RunId]),{t_name_full}[RunId])",
                format_string=int_format,
            )
        m_name = "Capacities"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS(DISTINCT({t_name_full}[Capacity]))"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Models"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS(DISTINCT({t_name_full}[Model]))"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Workspaces"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS(DISTINCT({t_name_full}[Workspace]))"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Violations"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS({t_name_full})"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Error Violations"
        if not any(m.Name == m_name for m in tom.all_measures()):
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=f'CALCULATE([Violations],{t_name_full}[Severity]="Error")',
                format_string=int_format,
            )
        m_name = "Rules Violated"
        if not any(m.Name == m_name for m in tom.all_measures()):
            calc = f"COUNTROWS(DISTINCT({t_name_full}[Rule Name]))"
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=get_expr(t_name_full, calc),
                format_string=int_format,
            )
        m_name = "Rule Severity"
        if not any(m.Name == m_name for m in tom.all_measures()):
            tom.add_measure(
                table_name=t_name,
                measure_name=m_name,
                expression=f"IF(ISFILTERED({t_name_full}[Rule Name]),IF( HASONEVALUE({t_name_full}[Rule Name]),MIN({t_name_full}[Severity])))",
            )
        # tom.add_measure(table_name=t_name, measure_name='Rules Followed', expression="[Rules] - [Rules Violated]")

    # Refresh the model
    refresh_semantic_model(dataset=dataset, workspace=lakehouse_workspace_id)


StatementMeta(, 30d88d90-4800-4e92-92f1-49c9ba35c7de, 13, Finished, Available, Finished)